##  TripAdvisor_Rating [Brusova Galina]
---

### Импорт библиотек

In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor 
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import datetime
import os
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import re
import math
import copy
from IPython.display import display
pd.options.mode.chained_assignment = None

In [ ]:
RANDOM_SEED = 42
!pip freeze > requirements.txt

### Импорт данных

In [ ]:
path_to_file = '/kaggle/input/sf-dst-restaurant-rating/'
path_to_file_2 = '/kaggle/input/new-city/'
path_to_file_3 = '/kaggle/input/city-ch/'
df_train = pd.read_csv(path_to_file+'main_task.csv')
df_test = pd.read_csv(path_to_file+'kaggle_task.csv')
cityn = pd.read_excel(path_to_file_2+'cityn.xls')
city_ch = pd.read_excel(path_to_file_3+'city_ch.xls')
pd.set_option('display.max_columns', 200)
display(df_train.head(2))
display(df_test.head(2))
display(cityn.head(2))
display(city_ch.head(2))

Значение колонок таблицы City (данные взяты с сайта https://www.numbeo.com/):
* city - название города
* sallary - Average Monthly Net Salary (After Tax)
* ipoteka - Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate	
* apart - Apartment (1 bedroom) in City Centre
* transport - One-way Ticket (Local Transport)
* milk_cost - Milk (regular), (1 liter)
* exp_rest - cost Meal, Inexpensive Restaurant
* mid_rest - Meal for 2 People, Mid-range Restaurant, Three-course
* mcdac - McMeal at McDonalds (or Equivalent Combo Meal)
* turizm - count of tourists

Значение колонок таблицы City_ch (данные взяты с сайта https://www.numbeo.com/):
* Crime - уровень преступности
* Edu - уровень обоазования
* Health - коэффициент уровня медицины 

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['Sample'] = 1 # помечаем где у нас трейн
df_test['Sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
df = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

# Детальный анализ по переменным

## 1. Обработаем даты отзывов

In [ ]:
df.Reviews = df.Reviews.apply(lambda string: np.nan if string == '[[], []]' else string)
df[['g1', 'g2', 'g3', 'g4', 'g5']] = df.Reviews.str.split("[", expand=True)
del df['g1']
del df['g2']
del df['g5']
del df['g3']
df1 = df[['l1', 'l2', 'l3', 'l4', 'l5']] = df.g4.str.split("'", expand=True)
df1.columns = ['genres1', 'genres2', 'genres3', 'genres4', 'genres5']

data['date1'] = df['l2']
data['date2'] = df['l4']


def change_max(column):
    # присвоим значениям NaN самое частое значение
    return data[column].fillna((data[column].value_counts().idxmax()), inplace=True)

change_max('date1')
change_max('date2')

display(data)

## 2. Найдем разницу между годами первого и второго отзыва

In [ ]:
data[['m1', 'd1', 'Y1']] = data.date1.str.split("/", expand=True)
data[['m2', 'd2', 'Y2']] = data.date2.str.split("/", expand=True)
for i in data['Y1']:
    str(i).replace(' ', '')
data['Y1'] = data['Y1'].replace(r'\s+', '', regex=True)
change_max('Y1') # заменим пустые значения Y1 на максимально встречающиеся 
change_max('Y2') # заменим пустые значения Y2 на максимально встречающиеся 
data['Y1'] = data['Y1'].astype(int) # изменим формат на int
data['Y2'] = data['Y2'].astype(int) # изменим формат на int
data['razn_Y'] = data['Y2']-data['Y1'] # найдем разницу между первым и вторым годом отзыва
display(data)

## 3. Добавим признак сетевого ресторана

In [ ]:
## считаем, что если ID ресторана одинаковый, ресторан является сетевым
chain_lst = list(data.Restaurant_id.value_counts()[data.Restaurant_id.value_counts() > 1].index)
data['chain'] = data[data.Restaurant_id.isin(chain_lst)].Restaurant_id.apply(lambda x: 1)
data['chain'].fillna(0, inplace=True)
data['chain'].value_counts()

## 4. Добавим рейтинг городов (City_rate) по уровню жизни (данные wiki) 

In [ ]:
df = data
cityk = pd.Series([41, 39, 46, 43, 13, 41, 56, 69, 37, 1, 11, 28, 19, 3, 40, 23, 76, 82, 33, 8, 89, 45, 2, 38, 9, 82, 25, 31, 80, 18, 74], index=['London', 'Paris', 'Madrid', 'Barcelona', 'Berlin', 'Milan', 'Rome', 'Prague', 'Lisbon', 'Vienna',
'Amsterdam', 'Brussels', 'Hamburg', 'Munich', 'Lyon', 'Stockholm', 'Budapest', 'Warsaw', 'Dublin', 'Copenhagen', 'Athens', 'Edinburgh',
'Zurich', 'Oporto', 'Geneva', 'Krakow', 'Oslo', 'Helsinki', 'Bratislava', 'Luxembourg', 'Ljubljana']).rename_axis('City')

cityk= cityk.reset_index() # переиндексируем строки
cityk.columns = ['City','city_rate'] # даем названия столбцам
data = data.merge(cityk, on='City', how='left') # проводим merge с основной таблицей данных
display(data)

##  5. Добавим колонку суммы ресторанов по городам (City_count)

In [ ]:
rest_count=df['City'].value_counts()
rest_count = rest_count.reset_index() # переиндексируем строки
rest_count.columns = ['City','City_count'] # дадим названия колонкам


data = data.merge(rest_count, on='City', how='left') # проводим merge с основной таблицей данных
display(data)

## 6. Сделаем merge основной таблицы с таблицей Cityn и City_ch с данными по городам

In [ ]:
data = data.merge(cityn, on='City', how='left')
data = data.merge(city_ch, on='City', how='left')
display(data)

## 7. Добавим признак (city_ranking) ranking с учетом количества ресторанов в городе

In [ ]:
data['city_ranking']=data['City_count']/data['Ranking']
display(data)

## 8. Заменим данные с нулевыми значениями

In [ ]:
def change_max(column):
    # присвоим значениям NaN самое частое значение
    return data[column].fillna((data[column].value_counts().idxmax()), inplace=True)

def change_mode(column):
    # присвоим значениям NaN значение моды
    return df[column].fillna((int(df[column].mode())), inplace=True)

change_max('Number of Reviews')
change_max('Cuisine Style')
change_max('Price Range')

## 9. Добавляем параметры по типам кухни и колонку (cuis_count) с количеством типов кухни в ресторане
* CuisVeg - колонка наличия вегетарианской кухни
* CuisBar - колонка наличия бара
* CuisAsia - колонка наличия азиатской кухни
* CuisEuro - колонка наличия европейской кухни

In [ ]:
data['CuisCoint'] = data['Cuisine Style'].str.count(',') +1 # количество кухонь в ресторане
data['CuisVeg'] = data['Cuisine Style'].str.count('Vegetarian Friendly' or 'Vegan')
data['CuisBar'] = data['Cuisine Style'].str.count('Bar' or 'Pub')
data['CuisAsia'] = data['Cuisine Style'].str.count('Asian' or 'Japanese')

data['CuisEuro'] = data['Cuisine Style'].str.count('European')

# попвытка проанализировать отзывы на предмет положительных слов дала ухудшение MAE.
#data['Smile'] = data['Reviews'].str.count('Good') + data['Reviews'].str.count('Best') + data['Reviews'].str.count('Nice')+ data['Reviews'].str.count('Wonderful')+ data['Reviews'].str.count('Fine')+data['Reviews'].str.count('good') + data['Reviews'].str.count('best') + data['Reviews'].str.count('nice')+ data['Reviews'].str.count('wonderful')+ data['Reviews'].str.count('fine')



## 10. Заменим данные стоимости на числовые параметры

In [ ]:
data['Price Range'] = data['Price Range'].replace('$', 1)
data['Price Range'] = data['Price Range'].replace('$$ - $$$', 2.5)
data['Price Range'] = data['Price Range'].replace('$$$$', 4)
display(data)

## 11. Добавим признак столицы

In [ ]:
list_Of_NotCapitalCity = ['Barcelona', 'Milan', 'Hamburg', 'Munich', 
                          'Lyon', 'Zurich', 'Oporto', 'Geneva', 'Krakow']
data['Capital_City'] = data['City'].apply(lambda x: 0.0 if x in list_Of_NotCapitalCity else 1.0)

## 12. Ranking нормализация

In [ ]:
data['Ranking']=data['Ranking']/1000

## 13. Посмотрим на данные в графическом виде

In [ ]:
smth = data.hist(figsize=(20, 20), bins=100)

In [ ]:
def rasp(column):
    # посмотрим распределение значений
    return pd.DataFrame(data[column].value_counts())

In [ ]:
display(rasp('CuisCoint'))

In [ ]:
display(rasp('Number of Reviews'))

In [ ]:
plt.plot(data['Number of Reviews'])

## 14. Построчная верификация первых двух строк

In [ ]:
display(data.head(2))

## 15. Удалим не числовые столбцы

In [ ]:
#df.drop(['Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA', 'date_of_Review', 'len_date', 'Сountry', 'Сity_population', 'mean_Ranking_on_City', 'count_Restorant_in_City', 'max_Ranking_on_City', ], axis=1, inplace=True, errors='ignore')
del data['City']
del data['Cuisine Style']
del data['Reviews']
del data['URL_TA']
del data['ID_TA']
del data['date1']
del data['date2']
del data['m1'] 
del data['m2'] 
del data['d1'] 
del data['d2'] 
#del data['Y1'] 
#del data['Y2'] 
del data['Crime'] 
del data['HealthCare']

In [ ]:
display(data)

# Разбиваем датасет на тренировочный и тестовый

In [ ]:
train_data = data.query('Sample == 1').drop(['Sample'], axis=1)
test_data = data.query('Sample == 0').drop(['Sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Restaurant_id', 'Rating'], axis = 1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Обучаем модель, генерируем результат и сравниваем с тестом

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)


# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# функция стандартного математического округления
def classic_round(d_num):
    return int(d_num + (0.5 if d_num > 0 else -0.5))

# функция округления кратно 0.5
def my_round(d_pred):
    result = classic_round(d_pred*2)/2
    if result <=5:
        return result
    else:
        return 5
    
# создание функции для векторов np
my_vec_round = np.vectorize(my_round)

In [ ]:
y_pred = my_vec_round(y_pred)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
MAE = metrics.mean_absolute_error(y_test, y_pred)
print('MAE:', MAE)

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (12,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Проверяем корреляцию важных переменных

In [ ]:
df_temp = data.loc[df['Sample'] == 1, list(feat_importances.nlargest(15).index[0:15])]
plt.rcParams['figure.figsize'] = (12,6)
ax = sns.heatmap(df_temp.corr(), annot=True, fmt='.2g')
i, k = ax.get_ylim()
ax.set_ylim(i+0.5, k-0.5)

Можно проследить несколько зависмсостей: 
1. Чем выше зарпалата в городе, тем выше ипотека
2. Чем больше количество отзывов, тем ниже оценка и большее количество видов кухни в ресторане.
3. Цена за апартаменты выше, чем более туристический город. Туризм также сильно влияет на Ranking.



In [ ]:
list_temp = list(feat_importances.nlargest(15).index[[9,10]])
display(df_temp[list_temp].corr())

In [ ]:
list_temp = list(feat_importances.nlargest(15).index[[0,1,6,10]])
df_temp[list_temp].corr()

# Submission

In [ ]:
sample_submission = pd.read_csv(path_to_file+'sample_submission.csv')


## Готовим submission

In [ ]:
test_data.info()

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating','Restaurant_id'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
# приведем к кратности шага 0.5
predict_submission = list(map(lambda x: round(x * 2)/2, predict_submission))

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)